# Day7_1: 동적 스크레이핑 (Selenium & WebDriver)

## 📚 학습 목표

**Part 1: 기초**
1. JavaScript 렌더링 페이지의 특징 이해하기
2. Selenium WebDriver 설정 및 브라우저 제어하기
3. 다양한 방법으로 웹 요소 찾기 (By.ID, CLASS_NAME, XPATH)
4. 클릭, 입력, 폼 제출 자동화하기
5. 대기 전략으로 안정적인 크롤링 구현하기

**Part 2: 심화**
1. 스크롤 처리로 무한 스크롤 페이지 수집하기
2. iframe과 팝업 전환 처리하기
3. 로그인 자동화 구현하기
4. headless 모드로 백그라운드 실행하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| JavaScript 렌더링 | requests로 못 가져오는 동적 콘텐츠 수집 | SNS 피드, 실시간 차트, 검색 결과 |
| Selenium | 브라우저 자동화로 복잡한 인터랙션 재현 | 로그인, 페이지 전환, 폼 제출 |
| 대기 전략 | 요소 로딩 시간 고려한 안정적 수집 | AJAX 로딩, 이미지 지연 로딩 |
| headless 모드 | 서버 환경에서 백그라운드 자동화 | 정기 수집 스크립트, CI/CD 파이프라인 |

**분석가 관점**: requests로 수집한 HTML이 텅 비어있다면, 그건 JavaScript가 콘텐츠를 렌더링하기 때문입니다. Selenium은 실제 브라우저를 제어하여 이 문제를 해결합니다!

---

# Part 1: 기초

---

## 1.1 정적 vs 동적 페이지 이해하기

### 정적 페이지 vs 동적 페이지

웹 페이지는 크게 두 종류로 나뉩니다.

**정적 페이지 (Static Page)**:
- 서버가 완성된 HTML을 바로 응답
- `requests`로 수집 가능
- 예: 뉴스 기사, 블로그 포스트, 위키피디아

**동적 페이지 (Dynamic Page)**:
- JavaScript가 브라우저에서 콘텐츠를 렌더링
- `requests`로 가져온 HTML에는 데이터가 없음
- 예: SNS 피드, 실시간 주가 차트, 검색 엔진 결과

**확인 방법**:
1. 웹 페이지에서 마우스 우클릭 → "페이지 소스 보기"
2. 보고 싶은 데이터가 소스에 있으면 **정적**, 없으면 **동적**

In [1]:
# 정적 페이지 예시 (requests로 수집 가능)
import requests
from bs4 import BeautifulSoup

# 위키피디아 페이지
url = "https://en.wikipedia.org/wiki/Python_(programming_language)"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 제목 추출
title = soup.find('h1', class_='firstHeading').text
print(f"페이지 제목: {title}")

ModuleNotFoundError: No module named 'requests'

In [ ]:
# 동적 페이지 예시 (requests로는 데이터 없음)
# 예: 인스타그램, 트위터 피드 등
# requests로 가져오면 <div id="root"></div> 같은 빈 껍데기만 보임
# → Selenium 필요!

### 💡 실무 예시: 뉴스 사이트 vs SNS

**정적 페이지 (네이버 뉴스)**:
- 기사 본문이 HTML에 포함됨
- requests + BeautifulSoup로 충분

**동적 페이지 (트위터 타임라인)**:
- 스크롤할 때마다 JavaScript가 새 트윗 로드
- Selenium으로 스크롤 + 대기 필요

---

## 1.2 Selenium WebDriver 설정

### Selenium이란?

- **실제 브라우저를 자동으로 제어**하는 도구
- 클릭, 입력, 스크롤, 페이지 전환 등 모든 동작 재현
- Chrome, Firefox, Edge 등 지원

In [ ]:
# Selenium 설치 (터미널에서)
# uv add selenium

# ChromeDriver는 자동 다운로드됨 (Selenium 4.6+)

In [4]:
# 1. WebDriver 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By

# 2. 브라우저 옵션 설정 (선택사항)
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 브라우저 창 숨기기
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# 3. 드라이버 생성
driver = webdriver.Chrome(options=options)

print("✅ WebDriver 설정 완료!")

✅ WebDriver 설정 완료!


In [5]:
# 4. 페이지 열기
driver.get("https://www.python.org")
print(f"현재 URL: {driver.current_url}")
print(f"페이지 제목: {driver.title}")

현재 URL: https://www.python.org/
페이지 제목: Welcome to Python.org


In [ ]:
# 5. 브라우저 종료 (필수!)
# driver.quit()  # 나중에 실행

### 💡 실무 예시: 컨텍스트 매니저로 자동 종료

실무에서는 `try-finally` 또는 컨텍스트 매니저를 사용해 반드시 브라우저를 종료합니다.

In [6]:
from selenium import webdriver

# 안전한 패턴
driver = webdriver.Chrome()
try:
    driver.get("https://www.python.org")
    print(driver.title)
finally:
    driver.quit()  # 오류 발생해도 반드시 종료
    print("✅ 브라우저 종료")

Welcome to Python.org
✅ 브라우저 종료


---

## 1.3 웹 요소 찾기 (Locator Strategies)

### By 클래스로 요소 찾기

Selenium은 다양한 방법으로 HTML 요소를 찾을 수 있습니다.

In [ ]:
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://www.python.org")

In [7]:
# 1. ID로 찾기
search_box = driver.find_element(By.ID, "id-search-field")
print(f"검색창 찾음: {search_box.tag_name}")

MaxRetryError: HTTPConnectionPool(host='localhost', port=52853): Max retries exceeded with url: /session/c211a73fd2c402ba189b82f0551982cf/element (Caused by NewConnectionError("HTTPConnection(host='localhost', port=52853): Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다"))

In [ ]:
# 2. CLASS_NAME으로 찾기
# 여러 개일 경우 find_elements (복수형)
links = driver.find_elements(By.CLASS_NAME, "tier-2")
print(f"tier-2 클래스 요소 개수: {len(links)}개")

In [ ]:
# 3. TAG_NAME으로 찾기
all_links = driver.find_elements(By.TAG_NAME, "a")
print(f"전체 링크 개수: {len(all_links)}개")

In [ ]:
# 4. CSS_SELECTOR로 찾기 (강력!)
nav_links = driver.find_elements(By.CSS_SELECTOR, "nav a")
print(f"네비게이션 링크: {len(nav_links)}개")

In [ ]:
# 5. XPATH로 찾기 (가장 유연)
# 텍스트 포함하는 요소 찾기
about_link = driver.find_element(By.XPATH, "//a[contains(text(), 'About')]")
print(f"About 링크: {about_link.text}")

### Locator 전략 선택 가이드

| 방법 | 사용 시기 | 예시 |
|------|----------|------|
| `By.ID` | ID가 있는 경우 (가장 빠름) | `id="search"` |
| `By.CLASS_NAME` | 클래스로 그룹화된 요소 | `class="product-card"` |
| `By.CSS_SELECTOR` | 복잡한 선택자 필요 시 | `div.container > ul > li` |
| `By.XPATH` | 텍스트, 위치 기반 탐색 | `//button[text()='Submit']` |

In [8]:
driver.quit()

### 💡 실무 예시: 뉴스 헤드라인 수집

네이버 뉴스 헤드라인을 CSS Selector로 수집해봅시다.

In [9]:
driver = webdriver.Chrome()
driver.get("https://news.naver.com/")

# 개발자 도구에서 확인한 헤드라인 선택자
headlines = driver.find_elements(By.CSS_SELECTOR, ".cjs_news_bl .cjs_t")

print("📰 뉴스 헤드라인 (상위 5개):")
for i, h in enumerate(headlines[:5], 1):
    print(f"{i}. {h.text}")

driver.quit()

📰 뉴스 헤드라인 (상위 5개):


---

## 1.4 클릭, 입력, 폼 제출 자동화

### 요소 상호작용

찾은 요소에 클릭, 입력 등의 동작을 수행할 수 있습니다.

In [10]:
driver = webdriver.Chrome()
driver.get("https://www.python.org")

# 1. 입력하기
search_box = driver.find_element(By.ID, "id-search-field")
search_box.send_keys("pandas")  # 텍스트 입력
print("✅ 'pandas' 입력 완료")

✅ 'pandas' 입력 완료


In [11]:
# 2. 클릭하기
search_button = driver.find_element(By.ID, "submit")
search_button.click()
print("✅ 검색 버튼 클릭")

✅ 검색 버튼 클릭


In [12]:
# 3. Enter 키 보내기
from selenium.webdriver.common.keys import Keys

driver.get("https://www.python.org")
search_box = driver.find_element(By.ID, "id-search-field")
search_box.send_keys("numpy")
search_box.send_keys(Keys.RETURN)  # Enter 키
print("✅ Enter로 검색")

✅ Enter로 검색


In [ ]:
# 4. 입력 필드 초기화
driver.get("https://www.python.org")
search_box = driver.find_element(By.ID, "id-search-field")
search_box.send_keys("잘못 입력")
search_box.clear()  # 내용 지우기
search_box.send_keys("올바른 검색어")
print("✅ 입력 초기화 후 재입력")

In [ ]:
driver.quit()

### 💡 실무 예시: 구글 검색 자동화

In [ ]:
import time

driver = webdriver.Chrome()
driver.get("https://www.google.com")

# 검색어 입력
search_box = driver.find_element(By.NAME, "q")
search_box.send_keys("python selenium tutorial")
search_box.send_keys(Keys.RETURN)

# 결과 로딩 대기
time.sleep(2)

# 검색 결과 제목 수집
results = driver.find_elements(By.CSS_SELECTOR, "h3")
print("🔍 검색 결과 (상위 5개):")
for i, r in enumerate(results[:5], 1):
    print(f"{i}. {r.text}")

driver.quit()

---

## 1.5 대기 전략 (Wait Strategies)

### 왜 대기가 필요한가?

동적 페이지에서는 요소가 즉시 로딩되지 않습니다:
- AJAX 요청으로 데이터 로드
- 이미지 지연 로딩
- JavaScript 애니메이션

**3가지 대기 방법**:
1. `time.sleep()` - 고정 시간 대기 (비추천)
2. `implicit_wait()` - 요소 찾을 때까지 최대 N초 대기
3. `explicit_wait()` - 특정 조건 만족할 때까지 대기 (추천)

In [ ]:
# 1. time.sleep() - 비추천
import time

driver = webdriver.Chrome()
driver.get("https://www.python.org")
time.sleep(3)  # 무조건 3초 대기 (비효율적)
driver.quit()

In [ ]:
# 2. implicit_wait() - 간단하지만 덜 정밀
driver = webdriver.Chrome()
driver.implicitly_wait(10)  # 최대 10초까지 요소 찾기 시도

driver.get("https://www.python.org")
# 요소가 바로 있으면 즉시 반환, 없으면 최대 10초 기다림
search_box = driver.find_element(By.ID, "id-search-field")
print("✅ 요소 찾음")

driver.quit()

In [ ]:
# 3. explicit_wait() - 가장 정밀 (추천)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://www.python.org")

# 특정 요소가 클릭 가능할 때까지 최대 10초 대기
wait = WebDriverWait(driver, 10)
search_box = wait.until(
    EC.element_to_be_clickable((By.ID, "id-search-field"))
)
print("✅ 검색창이 클릭 가능 상태")

search_box.send_keys("selenium")
driver.quit()

### 자주 쓰는 Expected Conditions

| 조건 | 의미 |
|------|------|
| `presence_of_element_located` | 요소가 DOM에 존재 |
| `visibility_of_element_located` | 요소가 화면에 보임 |
| `element_to_be_clickable` | 요소가 클릭 가능 |
| `text_to_be_present_in_element` | 특정 텍스트 포함 |
| `title_contains` | 페이지 제목에 텍스트 포함 |

### 💡 실무 예시: AJAX 로딩 대기

무한 스크롤 페이지에서 새 콘텐츠가 로드될 때까지 기다리기

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://example.com/infinite-scroll")

# 스크롤 후 새 아이템 로딩 대기
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

wait = WebDriverWait(driver, 10)
# 10개 이상의 아이템이 로드될 때까지 대기
wait.until(
    lambda d: len(d.find_elements(By.CLASS_NAME, "item")) > 10
)
print("✅ 새 콘텐츠 로드 완료")

driver.quit()

---

# Part 2: 심화

---

## 2.1 스크롤 처리

### 무한 스크롤 페이지 수집

SNS 피드, 검색 결과 등은 스크롤해야 새 콘텐츠가 로드됩니다.

In [ ]:
# 페이지 끝까지 스크롤
driver = webdriver.Chrome()
driver.get("https://www.python.org/downloads/")

# JavaScript로 스크롤 실행
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
print("✅ 페이지 끝까지 스크롤")

driver.quit()

In [13]:
# 특정 요소까지 스크롤
driver = webdriver.Chrome()
driver.get("https://www.python.org")

# 푸터 찾기
footer = driver.find_element(By.TAG_NAME, "footer")

# 요소가 보일 때까지 스크롤
driver.execute_script("arguments[0].scrollIntoView();", footer)
print("✅ 푸터까지 스크롤")

driver.quit()

✅ 푸터까지 스크롤


### 💡 실무 예시: 무한 스크롤 완전 수집

스크롤을 반복하며 모든 콘텐츠를 수집합니다.

In [ ]:
import time

driver = webdriver.Chrome()
driver.get("https://example.com/infinite-scroll")

last_height = driver.execute_script("return document.body.scrollHeight")
items = []

while True:
    # 현재 아이템 수집
    current_items = driver.find_elements(By.CLASS_NAME, "item")
    items.extend([item.text for item in current_items])
    
    # 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 로딩 대기
    
    # 새 높이 확인
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # 더 이상 새 콘텐츠 없음
    last_height = new_height

print(f"✅ 총 {len(items)}개 아이템 수집")
driver.quit()

---

## 2.2 iframe과 팝업 처리

### iframe 전환

iframe은 페이지 안의 또 다른 HTML 문서입니다. iframe 내부 요소에 접근하려면 **컨텍스트를 전환**해야 합니다.

In [ ]:
driver = webdriver.Chrome()
driver.get("https://example.com/page-with-iframe")

# 1. iframe으로 전환
iframe = driver.find_element(By.TAG_NAME, "iframe")
driver.switch_to.frame(iframe)
print("✅ iframe으로 전환")

# 2. iframe 내부 요소 조작
button = driver.find_element(By.ID, "submit")
button.click()

# 3. 메인 페이지로 복귀
driver.switch_to.default_content()
print("✅ 메인 페이지로 복귀")

driver.quit()

### 새 창/탭 전환

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.python.org")

# 원래 창 핸들 저장
main_window = driver.current_window_handle
print(f"메인 창: {main_window}")

# 새 탭에서 링크 열기 (Ctrl+클릭 시뮬레이션)
link = driver.find_element(By.LINK_TEXT, "Downloads")
link.click()

# 모든 창 핸들 가져오기
all_windows = driver.window_handles
print(f"전체 창: {len(all_windows)}개")

# 새 창으로 전환
for window in all_windows:
    if window != main_window:
        driver.switch_to.window(window)
        print(f"새 창 제목: {driver.title}")
        break

# 원래 창으로 복귀
driver.switch_to.window(main_window)
print("✅ 메인 창으로 복귀")

driver.quit()

### 💡 실무 예시: Alert 팝업 처리

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://example.com/page-with-alert")

# 버튼 클릭 시 alert 발생
button = driver.find_element(By.ID, "alert-button")
button.click()

# alert 대기 및 수락
wait = WebDriverWait(driver, 10)
alert = wait.until(EC.alert_is_present())
print(f"Alert 메시지: {alert.text}")
alert.accept()  # 확인 클릭 (또는 alert.dismiss()로 취소)
print("✅ Alert 처리 완료")

driver.quit()

---

## 2.3 로그인 자동화

### 로그인 폼 처리

많은 사이트에서 로그인 후에야 데이터를 볼 수 있습니다.

In [ ]:
# 예시: GitHub 로그인 (실제로 실행하지 마세요!)
driver = webdriver.Chrome()
driver.get("https://github.com/login")

# 1. 아이디 입력
username_field = driver.find_element(By.ID, "login_field")
username_field.send_keys("your_username")

# 2. 비밀번호 입력
password_field = driver.find_element(By.ID, "password")
password_field.send_keys("your_password")

# 3. 로그인 버튼 클릭
login_button = driver.find_element(By.NAME, "commit")
login_button.click()

# 4. 로그인 완료 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "user-profile")))
print("✅ 로그인 성공")

driver.quit()

### 💡 실무 예시: 쿠키 저장/로드로 재로그인 생략

한 번 로그인한 후 쿠키를 저장하면 다음 실행 시 재사용 가능합니다.

In [ ]:
import pickle

# 로그인 후 쿠키 저장
driver = webdriver.Chrome()
driver.get("https://github.com/login")
# ... 로그인 수행 ...

# 쿠키 저장
cookies = driver.get_cookies()
with open("github_cookies.pkl", "wb") as f:
    pickle.dump(cookies, f)
print("✅ 쿠키 저장 완료")

driver.quit()

In [ ]:
# 다음 실행 시 쿠키 로드
driver = webdriver.Chrome()
driver.get("https://github.com")

# 저장된 쿠키 로드
with open("github_cookies.pkl", "rb") as f:
    cookies = pickle.load(f)
    for cookie in cookies:
        driver.add_cookie(cookie)

# 페이지 새로고침하면 로그인 상태 유지
driver.refresh()
print("✅ 쿠키로 자동 로그인")

driver.quit()

---

## 2.4 headless 모드

### 브라우저 창 숨기기

실제 서버에서는 화면이 없으므로 headless 모드로 백그라운드 실행합니다.

In [ ]:
from selenium import webdriver

# headless 옵션 추가
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 창 숨기기
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

driver = webdriver.Chrome(options=options)
driver.get("https://www.python.org")
print(f"제목: {driver.title}")
print("✅ headless 모드로 실행 중 (화면 없음)")

driver.quit()

### User-Agent 변경

일부 사이트는 봇을 차단합니다. User-Agent를 변경하면 일반 브라우저처럼 보입니다.

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(options=options)
driver.get("https://httpbin.org/user-agent")
print(driver.page_source)  # User-Agent 확인

driver.quit()

### 💡 실무 예시: 정기 수집 스크립트

서버에서 cron으로 매일 실행하는 뉴스 수집 스크립트

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime

def collect_news():
    """헤드라인 수집 및 CSV 저장"""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    
    driver = webdriver.Chrome(options=options)
    driver.get("https://news.naver.com/")
    
    # 헤드라인 수집
    headlines = driver.find_elements(By.CSS_SELECTOR, ".cjs_news_bl .cjs_t")
    data = [{
        "timestamp": datetime.now(),
        "headline": h.text
    } for h in headlines]
    
    # CSV 저장
    df = pd.DataFrame(data)
    df.to_csv("news_headlines.csv", mode='a', index=False, header=False)
    
    driver.quit()
    print(f"✅ {len(data)}개 헤드라인 수집 완료")

# collect_news()  # 실행

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. WebDriver 설정 ⭐

**문제**: Selenium WebDriver를 설정하고 `https://www.python.org`를 열어 페이지 제목을 출력하세요.

**기대 결과**: `"Welcome to Python.org"`

In [ ]:
# 여기에 코드를 작성하세요


### Q2. 요소 찾기 ⭐⭐

**문제**: `https://www.python.org`에서 검색창 요소를 ID로 찾아 "pandas"를 입력하세요.

**힌트**: 검색창 ID는 `"id-search-field"`입니다.

In [ ]:
# 여기에 코드를 작성하세요


### Q3. CSS Selector로 여러 요소 수집 ⭐⭐

**문제**: `https://www.python.org`에서 네비게이션 링크를 CSS Selector `"nav a"`로 찾아 개수를 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. implicit_wait 사용 ⭐⭐⭐

**문제**: `implicit_wait(10)`을 설정하고 `https://www.python.org`에서 검색창을 찾아 "selenium"을 입력한 후 Enter 키를 보내세요.

**힌트**: `Keys.RETURN` 사용

In [ ]:
# 여기에 코드를 작성하세요


### Q5. explicit_wait 사용 ⭐⭐⭐

**문제**: `WebDriverWait`와 `EC.element_to_be_clickable`을 사용하여 검색창이 클릭 가능할 때까지 기다린 후 "numpy"를 입력하세요.

In [ ]:
# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. 스크롤 실행 ⭐⭐⭐

**문제**: `https://www.python.org`를 열고 JavaScript로 페이지 끝까지 스크롤하세요.

**힌트**: `driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")`

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 특정 요소까지 스크롤 ⭐⭐⭐⭐

**문제**: `https://www.python.org`에서 푸터(`<footer>` 태그)를 찾아 해당 요소까지 스크롤하세요.

**힌트**: `scrollIntoView()` 사용

In [ ]:
# 여기에 코드를 작성하세요


### Q8. headless 모드 설정 ⭐⭐⭐⭐

**문제**: headless 모드로 WebDriver를 설정하고 `https://www.python.org`의 제목을 출력하세요.

**요구사항**:
- `--headless` 옵션 사용
- `--no-sandbox` 옵션 추가

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 새 창 전환 ⭐⭐⭐⭐

**문제**: `https://www.python.org`에서 "Downloads" 링크를 클릭하여 새 창을 열고, 새 창의 제목을 출력한 후 원래 창으로 복귀하세요.

**단계**:
1. 메인 창 핸들 저장
2. Downloads 링크 클릭
3. 새 창으로 전환
4. 새 창 제목 출력
5. 메인 창으로 복귀

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 종합: 뉴스 헤드라인 수집 파이프라인 ⭐⭐⭐⭐⭐

**문제**: 네이버 뉴스(`https://news.naver.com/`)에서 헤드라인을 수집하여 pandas DataFrame으로 만들고 CSV로 저장하세요.

**요구사항**:
1. headless 모드 사용
2. CSS Selector로 헤드라인 수집 (`.cjs_news_bl .cjs_t`)
3. 상위 10개만 수집
4. DataFrame 컬럼: `["순번", "헤드라인"]`
5. `news_headlines.csv`로 저장

**기대 결과**: CSV 파일에 10개 헤드라인 저장

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/문법 | 실무 활용 |
|------|----------------|----------|
| WebDriver 설정 | `webdriver.Chrome()`, `driver.get()` | 브라우저 자동화 시작 |
| 요소 찾기 | `By.ID`, `By.CLASS_NAME`, `By.CSS_SELECTOR`, `By.XPATH` | 동적 콘텐츠 타겟팅 |
| 상호작용 | `send_keys()`, `click()`, `clear()` | 폼 제출, 검색 자동화 |
| implicit_wait | `driver.implicitly_wait(10)` | 전역 대기 설정 |
| explicit_wait | `WebDriverWait`, `EC.element_to_be_clickable` | 조건부 정밀 대기 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| 스크롤 | `execute_script("window.scrollTo(...)")` | 무한 스크롤, 지연 로딩 |
| iframe 전환 | `switch_to.frame()`, `switch_to.default_content()` | 임베디드 콘텐츠 접근 |
| 새 창 전환 | `window_handles`, `switch_to.window()` | 팝업, 새 탭 처리 |
| headless 모드 | `options.add_argument('--headless')` | 서버 환경, 백그라운드 실행 |
| 쿠키 관리 | `get_cookies()`, `add_cookie()` | 로그인 세션 유지 |

### 💡 실무 팁

1. **대기 전략**: `time.sleep()` 대신 `WebDriverWait` + `EC` 사용 (더 빠르고 안정적)
2. **Locator 우선순위**: ID > CSS Selector > XPath 순으로 시도
3. **headless 디버깅**: 개발 시 headless 끄고 디버깅, 배포 시만 켜기
4. **에러 처리**: `try-except`로 `NoSuchElementException`, `TimeoutException` 처리
5. **브라우저 종료**: 항상 `driver.quit()`로 리소스 정리 (`try-finally` 사용)
6. **봇 탐지 회피**: User-Agent 변경, 랜덤 딜레이, 쿠키 관리
7. **스크롤 패턴**: 무한 스크롤은 높이 비교로 종료 조건 판단